In [1]:
import robot
import map
import solver
import utils
import worlds

import numpy as np
import matplotlib.pyplot as plt
%matplotlib qt

# unitTest 1: Move robot and check odometrey measurements

In [4]:
car = robot.robot()

#control inputs
dtheta = np.radians(20)
ds = 0.4

time = 0 #[s]
dt = 0.5 
timeFinal = 10

#init history loggers
hist_GT = np.array(car.pose)
hist_DR = hist_GT.copy()

#set graphics
fig , ax = worlds.setWorldMap()
graphic_GT_traj, = plt.plot([], [],'ko-',markersize = 5)
graphic_DR_traj, = plt.plot([], [],'ro-',markersize = 5)
ax.legend(["ground truth","dead reckoning"])

#run and plot simulation
xcurrent_DR = hist_GT.copy()
with plt.ion():
    while time <= timeFinal:
        meas_odom = car.moveAndMeasureOdometrey((ds,ds,dtheta))
        
        #dead reckoning integration
        xcurrent_DR[2] += meas_odom.dtheta
        xcurrent_DR[0] += meas_odom.dx * np.cos(xcurrent_DR[2])
        xcurrent_DR[1] += meas_odom.dy * np.sin(xcurrent_DR[2])
        
        #log history
        hist_DR = np.vstack([hist_DR,xcurrent_DR])
        hist_GT = np.vstack([hist_GT,car.pose])

        #increase time
        time += dt

        #plot
        graphic_GT_traj.set_data(hist_GT[:,0],hist_GT[:,1])
        graphic_DR_traj.set_data(hist_DR[:,0],hist_DR[:,1])
        plt.draw()
        plt.pause(dt)

# unitTest 2: Build prior map

In [2]:
np.random.seed(seed=2)

N = 10 #number of landmarks
semantics = ("table","MEP","chair","pillar","clutter")
xrange = (-2,2)
yrange = (-1,3)
sigMin = -0.5
sigMax = 0.5

landmarks = [None] * N
for ii in range(N):
    sigs = np.random.uniform(low=sigMin, high=sigMax, size=(2,2))
    cov = sigs@sigs.T #enforce symmetric and positive definite: https://mathworld.wolfram.com/PositiveDefiniteMatrix.html
    landmarks[ii] = map.landmark(x = (np.random.rand()-0.5) * np.diff(xrange) + np.mean(xrange),
                                 y = (np.random.rand()-0.5) * np.diff(yrange) + np.mean(yrange),
                                 classLabel = np.random.choice(semantics),
                                 cov = cov)

priorMap = map.map(landmarks)
priorMap.plot(plotIndex = True,plotCov = True)

# unitTest 3: Move car and check landmark measurements

In [12]:
#------Build worldmap
fig , ax = worlds.setWorldMap()
np.random.seed(seed=0)

N = 20 #number of landmarks
semantics = ("table","MEP","chair","pillar","clutter")
xrange = (-2,2)
yrange = (-1,3)

landmarks = [None] * N
for ii in range(N):
    landmarks[ii] = map.landmark(x = (np.random.rand()-0.5) * np.diff(xrange) + np.mean(xrange),
                                 y = (np.random.rand()-0.5) * np.diff(yrange) + np.mean(yrange),
                                 classLabel = np.random.choice(semantics),
                                 )
worldMap = map.map(landmarks)
worldMap.plot(ax = ax, plotIndex = True,plotCov = False)

#------Spawn Robot
car = robot.robot()
car.FOV = 2*np.pi
car.range = 1
#control inputs
ds = 0.4 #[m]
dtheta = np.radians(20)
odom = (ds,ds,dtheta)

#------Simulation parameters
time = 0 #[s]
dt = 0.5 
timeFinal = 10

#init history loggers
hist_GT = np.array(car.pose)
hist_DR = hist_GT.copy()
hist_LM = []

#set graphics
graphic_GT_traj, = plt.plot([], [],'ko-',markersize = 5)
graphic_DR_traj, = plt.plot([], [],'ro-',markersize = 5)

#run and plot simulation
xcurrent_DR = hist_GT.copy()
with plt.ion():
    while time <= timeFinal:
        meas_odom = car.moveAndMeasureOdometrey(odom)

        meas_lm = car.measureLandmarks(worldMap)

        #dead reckoning integration
        xcurrent_DR[2] += meas_odom.dtheta
        xcurrent_DR[0] += meas_odom.dx * np.cos(xcurrent_DR[2])
        xcurrent_DR[1] += meas_odom.dy * np.sin(xcurrent_DR[2])
        
        #log history
        hist_GT = np.vstack([hist_GT,car.pose])
        hist_DR = np.vstack([hist_DR,xcurrent_DR])
        hist_LM.append(meas_lm)

        #increase time
        time += dt

        #plot
        graphic_GT_traj.set_data(hist_GT[:,0],hist_GT[:,1])
        graphic_DR_traj.set_data(hist_DR[:,0],hist_DR[:,1])
        plt.draw()
        plt.pause(dt)

        print(meas_lm)


[<robot.meas_landmark object at 0x7f75906253d0>, <robot.meas_landmark object at 0x7f7590625d00>, <robot.meas_landmark object at 0x7f7590625e50>, <robot.meas_landmark object at 0x7f7590625bb0>, <robot.meas_landmark object at 0x7f7590625d30>]
[<robot.meas_landmark object at 0x7f7591485d00>, <robot.meas_landmark object at 0x7f7591464040>, <robot.meas_landmark object at 0x7f7591464640>, <robot.meas_landmark object at 0x7f7591464190>, <robot.meas_landmark object at 0x7f75913c8f70>]
[<robot.meas_landmark object at 0x7f7591464340>, <robot.meas_landmark object at 0x7f7591464160>, <robot.meas_landmark object at 0x7f759044a280>, <robot.meas_landmark object at 0x7f757fcaa280>, <robot.meas_landmark object at 0x7f75913c87f0>]
[<robot.meas_landmark object at 0x7f75913c8520>, <robot.meas_landmark object at 0x7f75914055b0>, <robot.meas_landmark object at 0x7f7591405e50>, <robot.meas_landmark object at 0x7f75914716d0>]
[<robot.meas_landmark object at 0x7f7591471970>, <robot.meas_landmark object at 0x7f

# unitTest4: Plot Robot Cone

In [14]:
#------Build worldmap
fig , ax = worlds.setWorldMap()
np.random.seed(seed=0)

#------Spawn Robot
car = robot.robot()
car.plot(ax)
#control inputs
ds = 0.4 #[m]
dtheta = np.radians(20)
odom = (ds,ds,dtheta)

#------Simulation parameters
time = 0 #[s]
dt = 0.5 
timeFinal = 10

with plt.ion():
    while time <= timeFinal:
        car.moveAndMeasureOdometrey(odom)

        #increase time
        time += dt

        car.plot()
        plt.draw()
        plt.pause(dt)

# unitTest5: putting everything together before gtsam

In [15]:
car, worldMap, ax, fig = worlds.default_world()
prms = worlds.default_parameters()

#init history loggers
hist_GT = np.array(car.pose)
hist_DR = hist_GT.copy()
hist_LM = []

#set graphics
graphic_GT_traj, = plt.plot([], [],'ko-',markersize = 1)
graphic_DR_traj, = plt.plot([], [],'ro-',markersize = 1)

#run and plot simulation
time = 0
xcurrent_DR = hist_GT.copy()
with plt.ion():
    while time <= prms["time"]["timeFinal"]:
        meas_odom = car.moveAndMeasureOdometrey(prms["odom"])
        meas_lms = car.measureLandmarks(worldMap)

        #dead reckoning integration
        xcurrent_DR[2] += meas_odom.dtheta
        xcurrent_DR[0] += meas_odom.dx * np.cos(xcurrent_DR[2])
        xcurrent_DR[1] += meas_odom.dy * np.sin(xcurrent_DR[2])

        #log history
        hist_GT = np.vstack([hist_GT,car.pose])
        hist_DR = np.vstack([hist_DR,xcurrent_DR])
        hist_LM.append(meas_lms)

        #increase time
        time += prms["time"]["dt"]

        #plot
        car.plot()
        graphic_GT_traj.set_data(hist_GT[:,0],hist_GT[:,1])
        graphic_DR_traj.set_data(hist_DR[:,0],hist_DR[:,1])
        ax.set_title([lm.index for lm in meas_lms])

        plt.draw()
        plt.pause(prms["time"]["dt"])

# unitTest6 GTSAM

notes: </br>
examples folder: /home/alon18/LocalInstalls/miniconda3/envs/smallslam/lib/python3.9/site-packages/gtsam/examples </br>
* imuFactorISAM2Example.py - understand difference between graph.add and graph.push_back </br>
  + never mind, they are the same (?) https://gtsam.org/doxygen/a03480.html#a1bd7154e74538b5934fc616bb5ef7281 </br>
* PlanarSLAMExample.py- factor for range and bearing </br>


from VisualISAMExample.py: </br>
      # If this is the first iteration, add a prior on the first pose to set the coordinate frame </br>
        # and a prior on the first landmark to set the scale </br>
        # Also, as iSAM solves incrementally, we must wait until each is observed at least twice before </br>
        # adding it to iSAM. </br>

In [16]:
car, worldMap, ax, fig = worlds.default_world()
prms = worlds.default_parameters()
backend = solver.solver(ax = ax, X0cov = car.odometry_noise)

car.odometry_noise = car.odometry_noise
car.rgbd_noise = car.rgbd_noise
car.FOV = np.pi/2

#init history loggers
hist_GT = np.array(car.pose)
hist_DR = hist_GT.copy()
hist_LM = []
hist_solverResults = []

#set graphics
graphic_GT_traj, = plt.plot([], [],'ko-',markersize = 1)
graphic_DR_traj, = plt.plot([], [],'ro-',markersize = 1)

#run and plot simulation
time = 0
xcurrent_DR = hist_GT.copy()
with plt.ion():
    while time <= prms["time"]["timeFinal"]*2:
        meas_odom = car.moveAndMeasureOdometrey((prms["odom"]))
        meas_lms = car.measureLandmarks(worldMap)

        backend.i += 1 #increase time index. Must be done before adding measurements as factors
        backend.addOdomMeasurement(meas_odom)
        for meas_lm in meas_lms:
            backend.addlandmarkMeasurement(meas_lm)
        backend.update(N=0)

        #dead reckoning integration
        xcurrent_DR[2] += meas_odom.dtheta
        xcurrent_DR[0] += meas_odom.dx * np.cos(xcurrent_DR[2])
        xcurrent_DR[1] += meas_odom.dy * np.sin(xcurrent_DR[2])
        
        #log history
        hist_GT = np.vstack([hist_GT,car.pose])
        hist_DR = np.vstack([hist_DR,xcurrent_DR])
        hist_solverResults.append(backend.current_estimate)

        #increase time
        time += prms["time"]["dt"]

        #plot
        car.plot()
        ax.set_title([lm.index for lm in meas_lms])
        backend.plot_landmarks()
        backend.plot_poses()

        graphic_GT_traj.set_data(hist_GT[:,0],hist_GT[:,1])
        graphic_DR_traj.set_data(hist_DR[:,0],hist_DR[:,1])
        
        plt.draw()
        plt.pause(prms["time"]["dt"])

TypeError: 'NoneType' object is not iterable

In [3]:
backend.ax

<AxesSubplot:title={'center':'[1, 11, 16, 17, 21, 24, 30, 35, 39]'}, xlabel='x', ylabel='y'>